In [58]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import json

import enum
import os
from git import Repo

In [50]:
repo_path = "./test_repo"

In [51]:
repo = Repo.clone_from("https://github.com/qcx/niamh", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/qcx/niamh ./test_repo
  stderr: 'fatal: destination path './test_repo' already exists and is not an empty directory.
'

In [ ]:
print([member for member in Language])

[<Language.CPP: 'cpp'>, <Language.GO: 'go'>, <Language.JAVA: 'java'>, <Language.KOTLIN: 'kotlin'>, <Language.JS: 'js'>, <Language.TS: 'ts'>, <Language.PHP: 'php'>, <Language.PROTO: 'proto'>, <Language.PYTHON: 'python'>, <Language.RST: 'rst'>, <Language.RUBY: 'ruby'>, <Language.RUST: 'rust'>, <Language.SCALA: 'scala'>, <Language.SWIFT: 'swift'>, <Language.MARKDOWN: 'markdown'>, <Language.LATEX: 'latex'>, <Language.HTML: 'html'>, <Language.SOL: 'sol'>, <Language.CSHARP: 'csharp'>, <Language.COBOL: 'cobol'>, <Language.C: 'c'>, <Language.LUA: 'lua'>, <Language.PERL: 'perl'>, <Language.HASKELL: 'haskell'>, <Language.ELIXIR: 'elixir'>, <Language.POWERSHELL: 'powershell'>]


In [ ]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".ts"],
    exclude=[
        "**/moderation.ts",
        "**/statistics.ts",
        "**/storedFilters.ts",
        "**/discursive/**",
        "**/trainer/**"
    ],
    parser = LanguageParser(language=Language.TS, parser_threshold=500)
)

documents = loader.load()
len(documents)


21

In [ ]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.TS, chunk_size=2000, chunk_overlap=200
)

texts = python_splitter.split_documents(documents)
len(texts)


30

In [ ]:
embeddings = OpenAIEmbeddings(api_key=os.getenv('Q_API_KEY'), disallowed_special=())

In [ ]:
db = Chroma.from_documents(texts, embeddings)

retriver = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8}
)

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    max_tokens=1000,
    openai_api_key=os.getenv('Q_API_KEY')
)

In [70]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseado"
        ),
        ("user", "{context}")
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriver, document_chain)


In [71]:
response = retrieval_chain.invoke({"input": "O que você acha do código abaixo?"})

In [72]:
import json

# Supondo que 'response' é o objeto retornado
def make_serializable(obj):
    if isinstance(obj, list):
        return [make_serializable(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: make_serializable(value) for key, value in obj.items()}
    elif hasattr(obj, "__dict__"):  # Verifica se é um objeto com atributos
        return make_serializable(obj.__dict__)
    elif hasattr(obj, "_asdict"):  # Para objetos do tipo namedtuple
        return make_serializable(obj._asdict())
    else:
        return obj

# Tornar serializável
serializable_response = make_serializable(response)

# Formatar como JSON
formatted_response = json.dumps(serializable_response, indent=4)
print(formatted_response)

{
    "input": "O que voc\u00ea acha do c\u00f3digo abaixo?",
    "context": [
        {
            "id": null,
            "metadata": {
                "language": "ts",
                "source": "test_repo/src/endpoints/qconcursos/discursive/services/generation-grade-stream.service.ts"
            },
            "page_content": "` -- 2) Coer\u00eancia e Coes\u00e3o (Nessa parte, verifique a coer\u00eancia e coes\u00e3o da minha resposta, como os conceitos foram apresentados e o que pode melhorar) \\n` +\n        ` -- 3) Dicas (Nessa parte, me d\u00ea dicas de como melhorar a minha resposta, quais conceitos deveria ter abordado, como poderia ter redigido de uma forma melhor e outros pontos que considerar relevante.) \\n` +\n        `- N\u00e3o fa\u00e7a nenhuma avalia\u00e7\u00e3o caso a resposta n\u00e3o tenha rela\u00e7\u00e3o com a pergunta ou seja ofensiva \\n` +\n        `- Utilize de tags HTML para facilitar a leitura da resposta \\n` +\n        `- N\u00e3o utilize de tags Mar

In [73]:
print(response['answer'])

A revisão do seu código envolve a análise da coerência e coesão das respostas, além de sugestões de melhorias. Vamos abordar essas áreas específicas em dois pontos principais: Coerência e Coesão, e Dicas.

### 1) Coerência e Coesão

**Coerência** se refere à lógica interna da resposta, enquanto **coesão** diz respeito à forma como as partes da resposta se conectam. 

- **Coerência**: Sua abordagem inicial é clara e direta, com instruções específicas sobre o que deve ser feito, como evitar avaliações de respostas irrelevantes ou ofensivas e a utilização de tags HTML. No entanto, a estrutura da mensagem poderia ser melhorada para apresentar os conceitos de forma mais sequencial e lógica. Por exemplo, você pode introduzir a necessidade de coerência e coesão logo após a descrição das instruções.

- **Coesão**: As partes da mensagem parecem bem conectadas, mas a transição entre as instruções e a parte em que você apresenta a resposta pode ser suavizada. Você poderia usar frases de transição